# Download and analyse authentication log events

## Downloading log events

We are going to analyse the log events of user `admin@example.com` (an admin/manager user).

To perform this task, you must have aws console access, because the keys are needed to access aws console via the cli.

Firstly though, we need to fetch the sub for this user.


In [12]:
user = "admin@example.com"

In [ ]:
# Run this command in the cli with the keys
# aws cognito-idp list-users --user-pool-id ap-southeast-2_3ZrrcagIG --filter "email = \"admin@example.com\"" --region ap-southeast-2

# Update region and the user pool id as required
# {
#     "Users": [
#         {
#             "Username": "admin@example.com",
#             "Attributes": [
#                 {
#                     "Name": "email",
#                     "Value": "admin@example.com"
#                 },
#                 {
#                     "Name": "email_verified",
#                     "Value": "true"
#                 },
#                 {
#                     "Name": "family_name",
#                     "Value": "Admin"
#                 },
#                 {
#                     "Name": "given_name",
#                     "Value": "Admin"
#                 },
#                 {
#                     "Name": "custom:terraform",
#                     "Value": "true"
#                 },
#                 {
#                     "Name": "custom:identity_id",
#                     "Value": "ap-southeast-2:099e873d-80b5-cb64-b9b4-0f64c663bd46"
#                 },
#                 {
#                     "Name": "sub",
#                     "Value": "f98e24c8-2011-70ae-9d93-084eb3f4b282"
#                 }
#             ],
#             "UserCreateDate": "2024-11-20T15:58:30.157000+10:30",
#             "UserLastModifiedDate": "2025-03-13T15:09:11.817000+10:30",
#             "Enabled": true,
#             "UserStatus": "CONFIRMED"
#         }
#     ]
# }

Note the following sub

```
f98e24c8-2011-70ae-9d93-084eb3f4b282
```


In [18]:
%%file download_auth_trails.sh

REGION="ap-southeast-2"

#!/bin/bash
aws cloudtrail lookup-events \
  --lookup-attributes AttributeKey=EventName,AttributeValue=InitiateAuth \
  --start-time 2025-05-01T00:00:00Z \
  --end-time 2025-05-22T23:59:59Z \
  --region $REGION \
  --output json > auth_trails.json

aws cloudtrail lookup-events \
  --lookup-attributes AttributeKey=EventName,AttributeValue=RevokeToken \
  --start-time 2025-05-01T00:00:00Z \
  --end-time 2025-05-22T23:59:59Z \
  --region $REGION \
  --output json > revoke_token_trails.json
  


Overwriting download_auth_trails.sh


In [15]:
# Run follwing command with keys in the terminal
# bash download_dataportal_logs.sh

## Loading the events and print them


In [16]:
import json
from pprint import pprint

with open("auth_trails.json", "r") as file:
    data = json.load(file)

for event in data["Events"]:
    cloudtrail_event = json.loads(event["CloudTrailEvent"])

    try:
        if not cloudtrail_event["additionalEventData"]["sub"] == "f98e24c8-2011-70ae-9d93-084eb3f4b282":
            continue


        if cloudtrail_event["requestParameters"]["authFlow"] == "USER_PASSWORD_AUTH":
            print(f'User password auth attempted by user "{cloudtrail_event["additionalEventData"]["sub"]}"({user}) at {cloudtrail_event["eventTime"]}')
    except KeyError:
        pass


User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-21T23:50:50Z
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-21T13:14:12Z
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-21T07:50:52Z
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-21T03:31:05Z
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-20T06:41:03Z
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-20T04:22:12Z
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-20T02:11:31Z
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-20T00:03:52Z
User password auth attempted by user "f98e24c8-2

In [28]:
import json
from pprint import pprint

with open("auth_trails.json", "r") as file:
    data = json.load(file)

for event in data["Events"]:
    cloudtrail_event = json.loads(event["CloudTrailEvent"])

    try:
        if not cloudtrail_event["additionalEventData"]["sub"] == "f98e24c8-2011-70ae-9d93-084eb3f4b282":
            continue


        if cloudtrail_event["requestParameters"]["authFlow"] == "USER_PASSWORD_AUTH":
            print(f'User password auth attempted by user "{cloudtrail_event["additionalEventData"]["sub"]}"({user}) at {cloudtrail_event["eventTime"]}')
            if error:= cloudtrail_event.get("errorMessage"):
                print(f'\tError: {error}')
            else:
                print(f'\tSucceeded')
    except KeyError:
        pass


User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-22T01:53:31Z
	Succeeded
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-22T01:53:06Z
	Succeeded
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-22T01:52:09Z
	Succeeded
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-22T01:51:10Z
	Succeeded
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-22T01:35:11Z
	Succeeded
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-22T01:20:47Z
	Succeeded
User password auth attempted by user "f98e24c8-2011-70ae-9d93-084eb3f4b282"(admin@example.com) at 2025-05-22T01:20:44Z
	Error: Incorrect username or password.
User password auth attempted by user "f98e24c8-2011-70ae-9d93